In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q git+https://github.com/openai/whisper.git
!apt update && apt install -y ffmpeg
!pip install nbstripout
!pip install OpenCC

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,065 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http:/

In [4]:
from opencc import OpenCC
import whisper
import json

In [5]:
folder_path = '/content/drive/My Drive/Ephesians_Book/'
audio_file_prefix = 'Eph'
text_file_prefix = audio_file_prefix
cc = OpenCC('t2s')
# Load the model (you can use "small", "medium", or "large" — large = best accuracy)
model = whisper.load_model("medium")  # medium is good for Chinese & faster than large

100%|██████████████████████████████████████| 1.42G/1.42G [00:11<00:00, 137MiB/s]


In [6]:
def file_name_builder(folder_path, file_prefix, file_extension, index):
  return f"{folder_path}{file_prefix}_{index:02d}.{file_extension}"

In [7]:
def convert_audio_files_to_text_json(start_index, end_index):
  for i in range(start_index, end_index):
    audio_file_name = file_name_builder(f"{folder_path}audio/",
                                        audio_file_prefix, "mp4", i)
    print(f"processing file {audio_file_name}")

    result = model.transcribe(audio_file_name, language="zh")
    # convert any traditional Chinese to simplified Chinese
    segment_object_list = transcription_result_to_objects(result)

    json_file_name = file_name_builder(f"{folder_path}text_output/",
                                      text_file_prefix, "json", i)
    with open(json_file_name, "w", encoding="utf-8") as f:
      json.dump(segment_object_list, f, ensure_ascii=False, indent=2)

In [8]:
def transcription_result_to_objects(result):
  segment_object_list = []
  for segment in result["segments"]:
    segment_object = {
        "content": cc.convert(segment["text"]),
        "startTime": segment["start"],
        "endTime": segment["end"],
        "titleIndex": None
    }
    segment_object_list.append(segment_object)
  return segment_object_list

In [9]:
convert_audio_files_to_text_json(59, 61)

processing file /content/drive/My Drive/Ephesians_Book/audio/Eph_59.mp4
processing file /content/drive/My Drive/Ephesians_Book/audio/Eph_60.mp4
